In [1]:
import pandas as pd

In [22]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

<h2> Q1 </h2>

In [2]:
df = pd.read_parquet("data/yellow_tripdata_2023-01.parquet")

In [3]:
df.shape

(3066766, 19)

There are 19 columns

<h2> Q2 </h2>

In [4]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [7]:
df["duration"] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [8]:
df.duration.describe()

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration, dtype: float64

<h2>Q3</h2>

In [12]:
((df.duration >= 1) & (df.duration <=60)).mean()

0.9812202822125979

98% of records are left after dropping outliers.

In [13]:
df = df[(df.duration >= 1) & (df.duration <=60)]

In [15]:
categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)
df[categorical].dtypes

PULocationID    object
DOLocationID    object
dtype: object

<h2>Q4</h2>

In [17]:
dv = DictVectorizer()

In [18]:
train_dict = df[categorical].to_dict(orient="records")

In [19]:
X_train = dv.fit_transform(train_dict)

In [20]:
X_train.shape

(3009173, 515)

There are 515 columns.

<h2>Q5</h2>

In [24]:
target = "duration"
y_train = df[target].values

In [25]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [26]:
y_pred = lr.predict(X_train)

In [27]:
mean_squared_error(y_train, y_pred, squared=False)

7.6492610279057605

<h2>Q6</h2>

In [28]:
df_val = pd.read_parquet("data/yellow_tripdata_2023-02.parquet")

In [29]:
df_val["duration"] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)

df_val = df_val[(df_val.duration >= 1) & (df_val.duration <=60)]

df_val[categorical] = df_val[categorical].astype(str)

In [ ]:
val_dict = df_val[categorical].to_dict(orient="records")

X_val = dv.transform(val_dict)
y_val = 